# Imports

In [26]:
import pandas as pd
from pathlib import Path
import os
import numba
from numba import jit
import numpy as np

# Obtenção dos dados

In [27]:
# Listar arquivos no diretório acima
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Definindo o caminho base para a pasta Pre-processamento
base_path = Path(parent_dir) / 'Pre-processamento'

file_path_raw = base_path / 'dataset_cotton_fruit.xlsx'
file_path_msc = base_path / 'data_msc.xlsx'
file_path_snv = base_path / 'data_snv.xlsx'
file_path_sg = base_path / 'data_savgol_ordem_1.xlsx'

df_raw = pd.read_excel(file_path_raw)
df_msc = pd.read_excel(file_path_msc)
df_snv = pd.read_excel(file_path_snv)
df_sg = pd.read_excel(file_path_sg)


# PLSR

In [28]:
X = df_raw.iloc[:, 7:].values
y = df_raw['SST'].values

In [29]:
# Code to implement the optimized version of the PLS Algorithm

import pandas as pd
import numpy as np
import numba
from numba import jit

@jit
def mean_center_scale(dataframe):
    '''Scale dataframe by subtracting mean and dividing by standard deviation'''
    dataframe = dataframe - dataframe.mean()
    dataframe = dataframe/dataframe.std()
    return dataframe

@jit
def y_pred(Y_pred, i,b_dictionary,t_hat_dictionary,q_new_dictionary):
    '''Find prediction for Y based on the number of components in this iteration'''
    for j in range(1,i+1):
        Y_pred = Y_pred + (b_dictionary[j]*t_hat_dictionary[j]).dot(q_new_dictionary[j].T)
    return Y_pred    

@jit
def rmse(i,Y_true, Y_pred, response_std, RMSE_dictionary):
    '''Find training RMSE''' 
    RMSE = np.sqrt(sum((Y_true - Y_pred)**2)/Y_true.shape[0])
    RMSE_scaled = RMSE * response_std 
    RMSE_dictionary[i] = RMSE_scaled

    return RMSE_dictionary

@jit        
def core_pls(i,Y, X, q_new_dictionary, b_dictionary, t_hat_dictionary) :
    '''Core PLS algorithm'''
    
    #Here we have one variable in the Y block so q = 1 
    #and omit steps 5-8
    q = 1

    #For the X block, u = Y
    u = Y #random y column from Y #Step 1
    w_old = np.dot(u.T,X)/np.dot(u.T,u) #Step 2
    w_new = w_old/np.linalg.norm(w_old) #Step 3
    t = np.dot(X,w_new.T)/np.dot(w_new,w_new.T) #Step 4

    #For the Y block can be omitted if Y only has one variable
    q_old = np.dot(t.T,Y)/np.dot(t.T,t) #Step 5
    q_new = q_old/np.linalg.norm(q_old) #Step 6
    q_new_dictionary[i] = q_new
    u = np.dot(Y,q_new.T)/np.dot(q_new,q_new.T) #Step 7

    #Step 8: Check convergence

    #Calculate the X loadings and rescale the scores and weights accordingly
    p = np.dot(t.T,X)/np.dot(t.T,t) #Step 9
    p_new = p.T/np.linalg.norm(p.T) #Step 10
    t_new = t/np.linalg.norm(p.T) #Step 11
    w_new = w_old/np.linalg.norm(p)  #Step 12

    #Find the regression coefficient for b for th inner relation
    b = np.dot(u.T,t_new)/np.dot(t.T,t) #Step 13
    b_dictionary[i] = b

    #Calculation of the residuals
    E_h = X - np.dot(t_new,p_new.T)
    F_h = Y - b.dot(t_new.T).T.dot(q) #WORKS BUT IS THIS RIGHT?        
    
    #Set outer relation for the X block
    #Xres_dictionary[i] = E_h #MAYBE REMOVE
    X = E_h
        
    #Set the mixed relation for the Y block
    #Yres_dictionary[i] = F_h 3MAYBE REMOVE
    Y = F_h
        
    #Find estimated t hat
    t_hat = np.dot(E_h,w_new.T)
    t_hat_dictionary[i] = t_hat
    E_h = E_h - np.dot(t_hat,p_new.T)
    
    return X,Y, u, w_new, q_new, t_new, p_new, q_new_dictionary, t_hat_dictionary, b_dictionary,E_h, F_h 
          

def pls_optimized(path, path_test, predictors, response):
    '''Function that takes a dataframe and runs partial least squares on numeric predictors for a numeric response.
    Returns the residuals of the predictor (X block), response (Y block), and traininig RMSE'''
    ###TRAINING DATA
    combined = predictors
    #Load data
    data = pd.DataFrame.from_csv(path)
    combined.append(response)
    data = data[combined]
    response_std = data[response].std()
    
    #Subtract the mean and scale each column
    data = mean_center_scale(data)

    #Separate in to design matrix (X block) and response column vector (Y block)
    predictors.pop()
    X = data[predictors].as_matrix()
    Y = data[[response]].as_matrix()
    Y_true = Y #For prediction
    
    #Get rank of matrix
    rank = np.linalg.matrix_rank(X)
   
    u = Y #set initial u as Y
    Xres_dictionary = {}
    Yres_dictionary = {}
    q_new_dictionary ={}
    b_dictionary = {}
    t_hat_dictionary = {}
    t_hat_train_dictionary = {}
    t_hat_test_dictionary = {}
    RMSE_dictionary = {}
    RMSE_test_dictionary = {}
    
    ###TEST DATA
    #Load data
    data_test = pd.DataFrame.from_csv(path_test)
    combined.append(response)
    data_test = data_test[combined]
    response_std_test = data_test[response].std()
    
    #Subtract the mean and scale each column
    data_test = mean_center_scale(data_test)

    #Separate in to design matrix (X block) and response column vector (Y block)
    predictors.pop()
    X_test = data[predictors].as_matrix()
    Y_test = data[[response]].as_matrix()
    Y_true_test = Y_test #For prediction
      
    #Get rank of matrix
    rank_test = np.linalg.matrix_rank(X_test)
    
    #Iterate through each component
    for i in range(1,(rank+1)):
        Y_pred = np.zeros((Y_true.shape[0],1))
        Y_pred_test = np.zeros((Y_true_test.shape[0],1))
        
        #Core algo
        X,Y, u, w_new, q_new, t_new, p_new, q_new_dictionary, t_hat_dictionary, b_dictionary,E_h, F_h = core_pls(i,Y, X, q_new_dictionary, b_dictionary, t_hat_dictionary)
                
        #NEW Sum over different compenents
        for g in range(1,i+1):
            t_hat_train = np.dot(E_h,w_new.T)
            t_hat_train_dictionary[g] = t_hat_train
            E_h = E_h - np.dot(t_hat_train, p_new.T)
            Y_pred = y_pred(Y_pred, g,b_dictionary,t_hat_dictionary,q_new_dictionary)
        
        #Find training RMSE 
        RMSE_dictionary = rmse(i,Y_true, Y_pred, response_std, RMSE_dictionary)
        
        #Set initial E_h as X_test data
        E_h_test = X_test
        
        #Sum over different compenents
        for k in range(1,i+1):
            t_hat_test = np.dot(E_h_test,w_new.T)
            t_hat_test_dictionary[k] = t_hat_test
            E_h_test = E_h_test - np.dot(t_hat_test, p_new.T)
            Y_pred_test = y_pred(Y_pred_test, k,b_dictionary,t_hat_test_dictionary,q_new_dictionary)
        
        #Find test RMSE 
        RMSE_test_dictionary = rmse(i,Y_true_test, Y_pred_test, response_std_test, RMSE_test_dictionary)
        
    return RMSE_dictionary, RMSE_test_dictionary



In [37]:
# Código corrigido da função pls
def pls(path, path_test, predictors, response):
    '''Function that takes a dataframe and runs partial least squares on numeric predictors for a numeric response.
    Returns the residuals of the predictor (X block), response (Y block), and training RMSE'''
    combined = predictors[:]  # Ready list to combine the predictors and response to get both sets of data
    
    # Data preparation for training data
    data = pd.read_csv(path)  # Retrieve full csv data from local machine
    combined.append(response)  # Add the response to the list of variables to get from data set
    data = data[combined]  # Only retrieve the predictors and response
    response_std = data[response].std()  # Store the response variable standard deviation to scale the RMSE to real units at end
    
    # Subtract the mean from each column
    data = data - data.mean()
    
    # Scale each column by the column standard deviation
    data = data / data.std()
    
    # Separate into design matrix (X block) and response column vector (Y block)
    predictors.remove(response)  # Remove the response variable from the predictors list
    X = data[predictors].to_numpy()  # Create a matrix of predictor values
    Y = data[[response]].to_numpy()  # Create a matrix of response values
    Y_true = Y.copy()  # Store the true Y values for prediction later
    
    # PLS algorithm
    u = Y.copy()  # Set initial u value as response variables
    Xres_dictionary = {}  # Create a dictionary for the residuals from the decomposition of the X block
    Yres_dictionary = {}  # Create a dictionary for the residuals from the decomposition of the Y block
    q_new_dictionary = {}  # Create a dictionary for row vectors of q loadings for the Y block
    b_dictionary = {}  # Create a dictionary for scalar regression coefficient for PLS components 
    t_hat_dictionary = {}  # Create a dictionary for the matrix of X scores 
    t_hat_train_dictionary = {}  # Create a dictionary for the matrix of X scores for training data
    t_hat_test_dictionary = {}  # Create a dictionary for the matrix of X scores for test data
    RMSE_dictionary = {}  # Create a dictionary to store RMSE for training data
    RMSE_test_dictionary = {}  # Create a dictionary to store RMSE for test data
    
    for i in range(1, (rank + 1)):
        Y_pred = np.zeros((Y_true.shape[0], 1))
        
        # Here we have one variable in the Y block so q = 1 and omit steps 5-8
        q = 1
        
        # For the X block, u = Y
        u = Y.copy()  # random y column from Y  # Step 1
        w_old = np.dot(u.T, X) / np.dot(u.T, u)  # Step 2
        w_new = w_old / np.linalg.norm(w_old)  # Step 3
        t = np.dot(X, w_new.T) / np.dot(w_new, w_new.T)  # Step 4
        
        # For the Y block can be omitted if Y only has one variable
        q_old = np.dot(t.T, Y) / np.dot(t.T, t)  # Step 5
        q_new = q_old / np.linalg.norm(q_old)  # Step 6
        q_new_dictionary[i] = q_new
        u = np.dot(Y, q_new.T) / np.dot(q_new, q_new.T)  # Step 7
        
        # Step 8: Check convergence
        
        # Calculate the X loadings and rescale the scores and weights accordingly
        p = np.dot(t.T, X) / np.dot(t.T, t)  # Step 9
        p_new = p.T / np.linalg.norm(p.T)  # Step 10
        t_new = t / np.linalg.norm(p.T)  # Step 11
        w_new = w_old / np.linalg.norm(p)   # Step 12
        
        # Find the regression coefficient for b for the inner relation
        b = np.dot(u.T, t_new) / np.dot(t.T, t)  # Step 13
        b_dictionary[i] = b
        
        # Calculation of the residuals
        E_h = X - np.dot(t_new, p_new.T)
        F_h = Y - b.dot(t_new.T).T.dot(q)
        
        # Set outer relation for the X block
        Xres_dictionary[i] = E_h
        X = E_h
        
        # Set the mixed relation for the Y block
        Yres_dictionary[i] = F_h
        Y = F_h
        
        # Find estimated t hat
        t_hat = np.dot(E_h, w_new.T)
        t_hat_dictionary[i] = t_hat
        E_h = E_h - np.dot(t_hat, p_new.T)
        
        # Predict training set response by summing over different components
        E_h = X.copy()
        for j in range(1, i + 1):
            t_hat_train = np.dot(E_h, w_new.T)
            t_hat_train_dictionary[j] = t_hat_train
            E_h = E_h - np.dot(t_hat_train, p_new.T)
            for g in range(1, i + 1):
                Y_pred = Y_pred + (b_dictionary[g] * t_hat_dictionary[g]).dot(q_new_dictionary[g].T)
        
        # Find training RMSE
        RMSE = np.sqrt(sum((Y_true - Y_pred)**2) / Y_true.shape[0])
        RMSE_scaled = RMSE * response_std
        RMSE_dictionary[i] = RMSE_scaled
        
        # Code chunk to find test RMSE
        # Load data
        data_test = pd.read_csv(path_test)
        combined.append(response)
        data_test = data_test[combined]
        response_std_test = data_test[response].std()
        
        # Subtract the mean from each column
        data_test = data_test - data_test.mean()
        
        # Scale each column by the column standard deviation
        data_test = data_test / data_test.std()
        
        # Separate into design matrix (X block) and response column vector (Y block)
        predictors.remove(response)
        X_test = data_test[predictors].to_numpy()
        Y_test = data_test[[response]].to_numpy()
        Y_true_test = Y_test.copy()  # For prediction
        
        Y_pred_test = np.zeros((Y_true_test.shape[0], 1))
        
        # Get rank of matrix
        rank_test = np.linalg.matrix_rank(X_test)
        
        E_h_test = X_test
        
        # Sum over different components
        for k in range(1, i + 1):
            t_hat_test = np.dot(E_h_test, w_new.T)
            t_hat_test_dictionary[k] = t_hat_test
            E_h_test = E_h_test - np.dot(t_hat_test, p_new.T)
            Y_pred_test = Y_pred_test + (b_dictionary[k] * t_hat_test_dictionary[k]).dot(q_new_dictionary[k].T)
        
        # Find test RMSE
        RMSE = np.sqrt(sum((Y_true_test - Y_pred_test)**2) / Y_true_test.shape[0])
        RMSE_scaled_test = RMSE * response_std_test
        RMSE_test_dictionary[i] = RMSE_scaled_test
        
    return RMSE_dictionary, RMSE_test_dictionary


In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Carregar o arquivo completo
df_raw = pd.read_excel(file_path_raw)

# Selecionar apenas a coluna 'SST' e as colunas da 7ª em diante
columns_to_keep = ['SST'] + list(df_raw.columns[7:])
df_selected = df_raw[columns_to_keep]

# Definir preditores e resposta
sim_predictors = df_selected.iloc[:, 1:].columns.tolist()  # Selecionar colunas a partir da 7ª até o final
sim_response = "SST"  # Nome da coluna de resposta

# Dividir os dados em treino e teste (70% treino, 30% teste)
train_df, test_df = train_test_split(df_selected, test_size=0.3, random_state=42)

# Salvar os dados de treino e teste em arquivos CSV
sim_data_path = 'sim_data_train.csv'
sim_data_test_path = 'sim_data_test.csv'
train_df.to_csv(sim_data_path, index=False)
test_df.to_csv(sim_data_test_path, index=False)

# Executar a regressão PLS nos dados simulados
pls_sim_results = pls(sim_data_path, sim_data_test_path, sim_predictors, sim_response)


KeyError: '[350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547